In [50]:

from keras.models import Model
from keras.layers import Input, Activation, concatenate
from keras.layers import Flatten, Dropout
from keras.layers import Conv2D, MaxPool2D
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import SGD

In [61]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils import to_categorical
from keras import backend as K

# Model
## SqueezeNet

In [60]:
# Refer from : https://github.com/rcmalli/keras-squeezenet/blob/master/keras_squeezenet/squeezenet.py

sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'
    x = Conv2D(squeeze, (1,1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)
    
    left = Conv2D(expand, (1,1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)
    
    right = Conv2D(expand, (3,3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)
    
    #x = add([left, right], name=s_id + 'concat')
    x = concatenate([left, right], axis=3, name=s_id + 'concat')
    
    return x
    

In [56]:
# Refer from : https://github.com/DT42/squeezenet_demo/blob/master/model.py

def squeezeNet(nb_classes, input_shape=(32,32,3)):
    inputs = Input((input_shape))
    x = Conv2D(96, (7,7), activation='relu', strides=(2,2), padding='same', name='conv1')(inputs)
    x = MaxPool2D(pool_size=(3,3), strides=(2,2), name='maxpool1')(x)
    
    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = MaxPool2D(pool_size=(3,3), strides=(2,2), name='maxpool2')(x)
    
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = MaxPool2D(pool_size=(3,3), strides=(2,2), name='maxpool3')(x)
    
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = Dropout(0.5, name='drop9')(x)
    
    x = Conv2D(nb_classes, (1,1), activation='relu', padding='valid', name='conv10')(x)
    x = GlobalAveragePooling2D(name='avgpool10')(x)
    
    softmax = Activation("softmax", name='softmax')(x)
    
    model = Model(inputs=inputs, outputs=softmax)
    model.summary()
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
           
    return model
    

In [63]:
model = squeezeNet(nb_classes=43)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_22 (InputLayer)            (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 16, 16, 96)    14208       input_22[0][0]                   
____________________________________________________________________________________________________
maxpool1 (MaxPooling2D)          (None, 7, 7, 96)      0           conv1[0][0]                      
____________________________________________________________________________________________________
fire2/squeeze1x1 (Conv2D)        (None, 7, 7, 16)      1552        maxpool1[0][0]                   
___________________________________________________________________________________________

In [64]:
model_checkpoint = ModelCheckpoint('model_TSR.hdf5', monitor='loss', save_best_only=True)
model_earlystopping = EarlyStopping(monitor='loss')

# Train and Test Model